In [3]:
import datetime
import pandas as pd
import time
import requests
import os
import numpy as np

In [132]:
## This class contains the components necessary for running and evaluating a historical simulation.

class SimulationEnv(object):
    
    def __init__(self, wealth, assets, 
                 path_to_data, start_date, end_date,
                 agent_type, expert_type):

        self.init_wealth = wealth
        self.wealth = wealth
        self.assets = assets
        self.agent_type = agent_type
        self.expert_type = expert_type
        self.path_to_data = path_to_data
        
        ## TODO: 
        ## - specify start and end date for simulation
        ## - collect statistics on simulation results
        
        return
    
    
    ## To be called before running a simulation - initialize experts, agents, and initial position, etc.
    def pre_simulation(self):
        
        self.experts = [Dummy(a, self.path_to_data+a.lower()+".csv") for a in self.assets]
        self.agent = self.agent_type(self.experts)
        self.positions = [weight * self.wealth for weight in self.agent.weights]
        
        ## TODO: 
        # do various other stuff here like for select for high-volatility stocks or something
        # exception handling
        # Need to make sure data files are in sync
        return
        
    ## Run simulation
    def run(self):
        while True:
            try:
                for expert in self.agent.experts:
                    expert.update()
                self.agent.update()
                self.positions = self.positions * (1 + self.agent.rewards)
                self.wealth = np.sum(self.positions)
                self.positions = [weight * self.wealth for weight in self.agent.weights]
            except StopIteration:
                break
                
                


In [133]:
## Base classes for agents and experts to be implemented by us

from abc import ABCMeta, abstractmethod

class Agent(object):
    __metaclass__ =ABCMeta
    
    @abstractmethod
    def __init__(self):
        pass
    
    @abstractmethod
    def update(self):
        pass
    
    @abstractmethod
    def act(self):
        pass
    
class Expert(object):
    __metaclass__=ABCMeta
    
    @abstractmethod
    def __init__(self, reward_data, context_data = None):
        pass
    
    @abstractmethod
    def update(self):
        pass
    

In [142]:
## Hedge. (http://www.cis.upenn.edu/~mkearns/finread/helmbold98line.pdf)

class Hedge(Agent):
    
    ## Initialize with a set of experts
    def __init__(self, experts):
        self.eta = -0.005
        self.experts = experts
        self.weights = np.ones(len(self.experts))/len(self.experts)
        self.rewards = None
        return
    
    ## Update the agent's rewards and its weights for each expert
    def update(self):
        self.rewards = np.asarray([e.reward for e in self.experts])
        multipliers = np.exp(self.eta * self.rewards/np.sum(self.weights * self.rewards))
        self.weights = (self.weights * multipliers)/ np.sum(self.weights * multipliers) 
        return
    
    def act(self):
        return self.weights

## Dummy expert that always pick the same asset
class Dummy(Expert):
    
    ## Expert has a reward associated with its pick
    def __init__(self, name, data):
        self.reward = 0.
        self.pick = name ## Might not be necessary
        self.data = pd.read_csv(data, iterator=True, chunksize=1)
        self.last_price = float(self.data.get_chunk(1)["adj_close"])
        return
    
    ## Expert updates its reward 
    def update(self):
        current_price = float(self.data.get_chunk(1)["adj_close"])
        self.reward = (current_price - self.last_price)/self.last_price
        self.last_price = current_price
        return

In [143]:
## Some other examples of agents to use as benchmarks 

class NaiveBuyHold(Agent):
    
    def __init__(self):
        return
    
    def update(self):
        return
    
    def act(self):
        return
    
class WeightedBuyHold(Agent):
    
    def __init__(self):
        return
    
    def update(self):
        return
    
    def act(self):
        return
    
class ConstantRebalancer(Agent):
    
    def __init__(self):
        return
    
    def update(self):
        return
    
    def act(self):
        return


In [139]:
stocks = ['AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DD', 'DIS', 'GE',
 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 
'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV', 'UNH', 'UTX', 'V', 'VZ', 'WMT', 'XOM']

In [140]:
s = SimulationEnv(100000, stocks, "data/djia_20150101_20171101/", "2015-01-01", "2017-11-01", Hedge, Dummy)
s.pre_simulation()
s.run()

In [ ]:
# DATA_DIR = "data/djia_20150101_20171101/"

# dummy_experts = [Dummy(s, DATA_DIR+s.lower()+".csv") for s in stocks]
# h = Hedge(dummy_experts)
# while True:
#     try:
#         for expert in h.experts:
#             expert.update()
#         h.update()
#         h.act()
#     except StopIteration:
#         break
# h.weights